<a href="https://colab.research.google.com/github/kesakeerthi/hello-world/blob/master/PySpark_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark General Usages

## Download and install Spark

In [0]:
!ls


drive		     sample_data		spark-2.4.4-bin-hadoop2.7.tgz
reported-crimes.csv  spark-2.4.4-bin-hadoop2.7	spark-2.4.4-bin-hadoop2.7.tgz.1


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Fetched 252 kB in 2s (139 kB/s)
Reading package lists... Done


## Setup environment

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data

In [0]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!ls -l

--2020-02-01 10:49:04--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

s.csv?accessType=DO     [             <=>    ]   1012M  3.22MB/s               ^C
total 1485816
-rw-r--r--  1 root root 1061275876 Feb  1 10:54 'rows.csv?accessType=DOWNLOAD'
drwxr-xr-x  1 root root       4096 Jan 13 16:38  sample_data
drwxr-xr-x 13 1000 1000       4096 Aug 27 21:30  spark-2.4.4-bin-hadoop2.7
-rw-r--r--  1 root root  230091034 Aug 27 22:01  spark-2.4.4-bin-hadoop2.7.tgz
-rw-r--r--  1 root root  230091034 Aug 27 22:01  spark-2.4.4-bin-hadoop2.7.tgz.1


In [0]:
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
!ls -l

total 1485816
-rw-r--r--  1 root root 1061275876 Feb  1 10:54 reported-crimes.csv
drwxr-xr-x  1 root root       4096 Jan 13 16:38 sample_data
drwxr-xr-x 13 1000 1000       4096 Aug 27 21:30 spark-2.4.4-bin-hadoop2.7
-rw-r--r--  1 root root  230091034 Aug 27 22:01 spark-2.4.4-bin-hadoop2.7.tgz
-rw-r--r--  1 root root  230091034 Aug 27 22:01 spark-2.4.4-bin-hadoop2.7.tgz.1


In [0]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') < lit('2018-11-11'))
rc.show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11| 

## Schemas

In [0]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [0]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|1153|
|0281|
|0620|
|0810|
|0281|
+----+
only showing top 5 rows



In [0]:
rc.select(col('IUCR')).show(5)

+----+
|IUCR|
+----+
|1153|
|0281|
|0620|
|0810|
|0281|
+----+
only showing top 5 rows



In [0]:
rc.select(rc.IUCR).show(5)

+----+
|IUCR|
+----+
|1153|
|0281|
|0620|
|0810|
|0281|
+----+
only showing top 5 rows



In [0]:
rc.select('Case Number', 'Date', 'Arrest').show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   JA366925|2001-01-01 11:00:00| false|
|   JB147188|2017-10-08 03:00:00| false|
|   JB147595|2017-03-28 14:00:00| false|
|   JB147230|2017-09-09 20:17:00| false|
+-----------+-------------------+------+
only showing top 4 rows



### Add new column name Ones with all entries 1 -- Literals

In [0]:
from pyspark.sql.functions import lit

In [0]:
rc.withColumn('Ones', lit(1)).show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+----+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|Ones|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+----+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|          

In [0]:
rc = rc.drop('IUCR')
rc.show(5)

+--------+-----------+-------------------+--------------------+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        

### Working with Rows

In [0]:
one_day = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') == lit('2018-11-11'))

In [0]:
one_day.count()

11

### Add the reported crimes for an additional day, 11-Nov-2018, to our dataset.

In [0]:
rc.union(one_day).orderBy('Date', ascending=False).show(15)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11507744|   JB516509|2018-11-11 00:00:00|   033XX S DAMEN AVE|0820|              THEFT|      $500 AND UNDER|           RESIDENCE|

### What are the top 10 number of reported crimes by Primary type, in descending order of occurence?

In [0]:
rc.groupby('Primary Type').count().orderBy('count', ascending = False).show(10)

+-------------------+------+
|       Primary Type| count|
+-------------------+------+
|              THEFT|870699|
|            BATTERY|783002|
|    CRIMINAL DAMAGE|487802|
|          NARCOTICS|431049|
|            ASSAULT|268251|
|      OTHER OFFENSE|265689|
|           BURGLARY|221129|
|MOTOR VEHICLE THEFT|198190|
| DECEPTIVE PRACTICE|167240|
|            ROBBERY|155657|
+-------------------+------+
only showing top 10 rows



### Challenge - 1: What percentage of reported crimes resulted in an arrest?

In [0]:
rc.select(rc.Arrest).distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [0]:
rc.groupby('Arrest').count().show()

+------+-------+
|Arrest|  count|
+------+-------+
| false|3033759|
|  true|1173699|
+------+-------+



In [0]:
rc.filter()

In [0]:
rc.filter(rc.Arrest == 'true').count() * 100 / rc.count()

27.895679529064818

### Challenge - 2: What are the top 3 locations for reported crimes?

In [0]:
rc.groupby('Location Description').count().orderBy('count', ascending = False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1154415|
|           RESIDENCE| 727307|
|           APARTMENT| 396006|
+--------------------+-------+
only showing top 3 rows



### Built-in Functions

In [0]:
from pyspark.sql import functions
print(dir(functions))

['Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_binary_mathfunctions', '_collect_list_doc', '_collect_set_doc', '_create_binary_mathfunction', '_create_column_from_literal', '_create_function', '_create_udf', '_create_window_function', '_functions', '_functions_1_4', '_functions_1_6', '_functions_2_1', '_functions_2_4', '_functions_deprecated', '_lit_doc', '_message', '_string_functions', '_test', '_to_java_column', '_to_seq', '_window_functions', '_wrap_deprecated_function', 'abs', 'acos', 'add_months', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'array_distinct', 'array_except', 'array_intersect', 'array_join', 'array_max', 'array_min', 'array_position', 'array_remove', 'array_repeat', 'array_sort', 'array_union', 'arrays_overlap', 'arrays_zip', 'asc', 'asc_null

### String Functions

Display Primary type - Lower, Upper, First 4 Chars

In [0]:
rc.select('Primary Type').show(10)

+--------------------+
|        Primary Type|
+--------------------+
|  DECEPTIVE PRACTICE|
| CRIM SEXUAL ASSAULT|
|            BURGLARY|
|               THEFT|
| CRIM SEXUAL ASSAULT|
| CRIM SEXUAL ASSAULT|
|OFFENSE INVOLVING...|
|OFFENSE INVOLVING...|
|               THEFT|
|  DECEPTIVE PRACTICE|
+--------------------+
only showing top 10 rows



In [0]:
from pyspark.sql.functions import lower, upper, substring
rc.select(lower(col('Primary Type'))).show(10)

+--------------------+
| lower(Primary Type)|
+--------------------+
|  deceptive practice|
| crim sexual assault|
|            burglary|
|               theft|
| crim sexual assault|
| crim sexual assault|
|offense involving...|
|offense involving...|
|               theft|
|  deceptive practice|
+--------------------+
only showing top 10 rows



In [0]:
rc.select(substring(col('Primary Type'),1,4)).show(10)

+-----------------------------+
|substring(Primary Type, 1, 4)|
+-----------------------------+
|                         DECE|
|                         CRIM|
|                         BURG|
|                         THEF|
|                         CRIM|
|                         CRIM|
|                         OFFE|
|                         OFFE|
|                         THEF|
|                         DECE|
+-----------------------------+
only showing top 10 rows



## Numeric Functions

Show Oldest and Recent date

In [0]:
from pyspark.sql.functions import min, max

rc.select(min(col('Date')), max(col('Date'))).show()

+-------------------+-------------------+
|          min(Date)|          max(Date)|
+-------------------+-------------------+
|2001-01-01 00:00:00|2018-11-10 23:55:00|
+-------------------+-------------------+



3 days earlier the  Oldest date and 1 month later the Recent date

In [0]:
from pyspark.sql.functions import date_add, date_sub
rc.select(  date_sub(min(col('Date')), 3), date_add(max(col('Date')),30 )         ).show()

+----------------------+-----------------------+
|date_sub(min(Date), 3)|date_add(max(Date), 30)|
+----------------------+-----------------------+
|            2000-12-29|             2018-12-10|
+----------------------+-----------------------+



## Working with Dates - Detail



In [0]:
from pyspark.sql.functions import to_date, to_timestamp, lit

In [0]:
# 1984-10-25 21:11:26
df = spark.createDataFrame( [ ('1984-10-25 21:11:26', ) ] , ['Birthday']       )
df.show()

+-------------------+
|           Birthday|
+-------------------+
|1984-10-25 21:11:26|
+-------------------+



In [0]:
df.select(to_date(col('Birthday') , 'yyyy-MM-dd HH:mm:ss'), to_timestamp(col('Birthday') , 'yyyy-MM-dd HH:mm:ss')).show(1)

+------------------------------------------+-----------------------------------------------+
|to_date(`Birthday`, 'yyyy-MM-dd HH:mm:ss')|to_timestamp(`Birthday`, 'yyyy-MM-dd HH:mm:ss')|
+------------------------------------------+-----------------------------------------------+
|                                1984-10-25|                            1984-10-25 21:11:26|
+------------------------------------------+-----------------------------------------------+



In [0]:
# 25/Oct/1984 21:11:26
df = spark.createDataFrame( [ ('25/Oct/1984 21:11:26', ) ] , ['Birthday']       )
df.select(to_date(col('Birthday') , 'dd/MMM/yyyy HH:mm:ss'), to_timestamp(col('Birthday') , 'dd/MMM/yyyy HH:mm:ss')).show(1)



+-------------------------------------------+------------------------------------------------+
|to_date(`Birthday`, 'dd/MMM/yyyy HH:mm:ss')|to_timestamp(`Birthday`, 'dd/MMM/yyyy HH:mm:ss')|
+-------------------------------------------+------------------------------------------------+
|                                 1984-10-25|                             1984-10-25 21:11:26|
+-------------------------------------------+------------------------------------------------+



In [0]:
# 10/25/1984 09:11:26 PM
df = spark.createDataFrame( [ ('10/25/1984 09:11:26 PM', ) ] , ['Birthday']       )
df.select(to_date(col('Birthday') , 'MM/dd/yyyy hh:mm:ss aa'), to_timestamp(col('Birthday') , 'MM/dd/yyyy hh:mm:ss aa')).show(1)

+---------------------------------------------+--------------------------------------------------+
|to_date(`Birthday`, 'MM/dd/yyyy hh:mm:ss aa')|to_timestamp(`Birthday`, 'MM/dd/yyyy hh:mm:ss aa')|
+---------------------------------------------+--------------------------------------------------+
|                                   1984-10-25|                               1984-10-25 21:11:26|
+---------------------------------------------+--------------------------------------------------+



## JOINS

In [0]:
!wget -O police-station.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD

--2020-02-01 16:25:48--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘police-station.csv’

police-station.csv      [ <=>                ]   5.57K  --.-KB/s    in 0s      

2020-02-01 16:25:49 (669 MB/s) - ‘police-station.csv’ saved [5699]



In [0]:
!ls


 drive				 sample_data
 police-station.csv		 spark-2.4.4-bin-hadoop2.7
 reported-crimes.csv		 spark-2.4.4-bin-hadoop2.7.tgz
'rows.csv?accessType=DOWNLOAD'	 spark-2.4.4-bin-hadoop2.7.tgz.1


In [0]:
ps = spark.read.csv('police-station.csv', header = True)
ps.show(5)

+------------+--------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|    DISTRICT| DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+------------+--------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|           1|       Central|     1718 S State St|Chicago|   IL|60616|http://home.chica...|312-745-4290|312-745-3694|312-745-3693| 1176569.052| 1891771.704|41.85837259|-87.62735617|(41.8583725929, -...|
|           6|       Gresham|   7808 S Halsted St|Chicago|   IL|60620|http://home.chica...|312-745-3617|312-745-3649|312-745-3639| 1172283.013| 1853022.646|41.75213684|-87.64422891|(41.752

In [0]:
rc.cache()
rc.count()

4207458

In [0]:
ps.select(col('DISTRICT')).distinct().show()

+------------+
|    DISTRICT|
+------------+
|           7|
|          15|
|          11|
|           3|
|           8|
|          22|
|          16|
|           5|
|          18|
|          17|
|           6|
|          19|
|          25|
|Headquarters|
|          24|
|           9|
|           1|
|          20|
|          10|
|           4|
+------------+
only showing top 20 rows



In [0]:
rc.select(col('DISTRICT')).distinct().show()

+--------+
|DISTRICT|
+--------+
|     009|
|     012|
|     024|
|    null|
|     031|
|     015|
|     006|
|     019|
|     020|
|     011|
|     025|
|     003|
|     005|
|     016|
|     018|
|     008|
|     022|
|     001|
|     014|
|     010|
+--------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import lpad

In [0]:
help(lpad)

Help on function lpad in module pyspark.sql.functions:

lpad(col, len, pad)
    Left-pad the string column to width `len` with `pad`.
    
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(lpad(df.s, 6, '#').alias('s')).collect()
    [Row(s='##abcd')]
    
    .. versionadded:: 1.5



In [0]:
ps.select(lpad(col('DISTRICT'),3,'0')).show(10)

+--------------------+
|lpad(DISTRICT, 3, 0)|
+--------------------+
|                 001|
|                 006|
|                 011|
|                 016|
|                 Hea|
|                 024|
|                 002|
|                 007|
|                 025|
|                 010|
+--------------------+
only showing top 10 rows



In [0]:
ps = ps.withColumn('Format_District',lpad(col('DISTRICT'),3,'0') )
ps.show(5)

+------------+--------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+---------------+
|    DISTRICT| DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|Format_District|
+------------+--------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+---------------+
|           1|       Central|     1718 S State St|Chicago|   IL|60616|http://home.chica...|312-745-4290|312-745-3694|312-745-3693| 1176569.052| 1891771.704|41.85837259|-87.62735617|(41.8583725929, -...|            001|
|           6|       Gresham|   7808 S Halsted St|Chicago|   IL|60620|http://home.chica...|312-745-3617|312-745-3649|312-745

In [0]:
rc.join(ps, rc.District == ps.Format_District, 'left_outer').show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------+--------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+---------------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|DISTRICT| DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|Format_District|
+--------+-----------+------------

## RDDs

In [0]:
psrdd = sc.textFile('police-station.csv')